# 🎯 Entraînement Senchess-AI sur Google Colab GPU

## ⚠️ Instructions importantes:

### 1. Activer le GPU
**AVANT** d'exécuter les cellules :
- `Runtime` → `Change runtime type` → `Hardware accelerator` → `GPU` (T4)
- Cliquez sur `Save`

### 2. Ordre d'exécution
Exécutez les cellules **dans l'ordre** :
1. ✅ Vérifier le GPU
2. ✅ Installer les dépendances
3. ✅ **Choisir UNE méthode** pour obtenir vos données (cellules 4, 5 OU 6)
4. ✅ Entraîner le modèle
5. ✅ Télécharger les résultats

### 3. Choix de la méthode de données
Exécutez **UNE SEULE** des cellules suivantes :
- **Cellule 4** : Cloner depuis GitHub (si votre projet est public)
- **Cellule 5** : Upload manuel d'un fichier .zip
- **Cellule 6** : Utiliser Google Drive

---

**Temps estimé** : 1-3 heures selon le dataset et les epochs


In [ ]:
# Vérifier le GPU disponible
!nvidia-smi

In [ ]:
# Installer les dépendances
!pip install ultralytics opencv-python pillow pyyaml

In [ ]:
# 📥 OPTION 1 : Cloner depuis GitHub
# ⚠️ Exécutez cette cellule OU la cellule 5 OU la cellule 6 (pas les 3 !)

!git clone https://github.com/MedouneSGB/Senchess-AI.git
%cd Senchess-AI

# Vérifier que les données sont présentes
import os
if os.path.exists('data/chess_ultimate_1693/data.yaml'):
    print("✅ Dataset trouvé ! Vous pouvez continuer.")
else:
    print("❌ Dataset non trouvé. Vérifiez que votre repo contient le dossier 'data'.")
    print("   Si les données ne sont pas sur GitHub, utilisez la cellule 5 ou 6 à la place.")

In [ ]:
# 📤 OPTION 2 : Upload manuel d'un fichier .zip
# ⚠️ Exécutez cette cellule OU la cellule 4 OU la cellule 6 (pas les 3 !)

from google.colab import files
import zipfile
import os

print("📦 Uploadez votre fichier .zip contenant le dataset...")
print("   Le fichier doit contenir : data/chess_ultimate_1693/data.yaml")
uploaded = files.upload()

# Décompresser
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"📂 Décompression de {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("✅ Décompression terminée !")

# Vérifier que le dataset est présent
if os.path.exists('data/chess_ultimate_1693/data.yaml'):
    print("✅ Dataset trouvé ! Vous pouvez continuer.")
else:
    print("❌ Fichier data.yaml non trouvé.")
    print("   Structure attendue : data/chess_ultimate_1693/data.yaml")
    print("   Vérifiez votre fichier .zip")

In [ ]:
# ☁️ OPTION 3 : Utiliser Google Drive
# ⚠️ Exécutez cette cellule OU la cellule 4 OU la cellule 5 (pas les 3 !)

from google.colab import drive
import os
import shutil

# Monter Google Drive
print("📁 Montage de Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive monté !")

# Copier les données depuis Drive
# ⚠️ MODIFIEZ le chemin selon où vous avez mis vos données dans Drive
source_path = '/content/drive/MyDrive/Senchess-AI/data'

if os.path.exists(source_path):
    print(f"📂 Copie depuis {source_path}...")
    shutil.copytree(source_path, './data', dirs_exist_ok=True)
    print("✅ Données copiées !")
else:
    print(f"❌ Chemin non trouvé : {source_path}")
    print("   Uploadez d'abord vos données sur Google Drive dans : MyDrive/Senchess-AI/data")
    print("   OU modifiez la variable source_path dans cette cellule")

# Vérifier que le dataset est présent
if os.path.exists('data/chess_ultimate_1693/data.yaml'):
    print("✅ Dataset trouvé ! Vous pouvez continuer.")
else:
    print("❌ Dataset non trouvé. Vérifiez le chemin dans votre Google Drive.")

In [ ]:
# 🚀 Entraîner le modèle avec GPU
# ⚠️ Assurez-vous d'avoir exécuté l'une des cellules précédentes pour charger les données

import os
from ultralytics import YOLO

# Vérifier que le fichier data.yaml existe
data_path = 'data/chess_ultimate_1693/data.yaml'
if not os.path.exists(data_path):
    raise FileNotFoundError(
        f"❌ Fichier non trouvé : {data_path}\n"
        "Veuillez d'abord exécuter l'une des cellules précédentes (4, 5 ou 6) "
        "pour charger vos données !"
    )

print("✅ Fichier data.yaml trouvé !")
print("🚀 Démarrage de l'entraînement...\n")

# Charger le modèle pré-entraîné
model = YOLO('yolov8n.pt')

# Entraîner (le GPU sera automatiquement détecté)
results = model.train(
    data=data_path,
    epochs=100,              # Nombre d'epochs (réduisez à 50 pour tester plus vite)
    imgsz=640,               # Taille des images
    batch=16,                # Augmentez à 32 ou 64 si vous avez assez de VRAM
    device=0,                # GPU 0 (automatique)
    name='senchess_colab_training',
    patience=20,             # Early stopping si pas d'amélioration
    save=True,               # Sauvegarder les checkpoints
    plots=True,              # Générer les graphiques
    cache=True,              # Cache les images pour accélérer
    workers=2                # Nombre de workers pour le chargement des données
)

print("\n✅ Entraînement terminé !")
print(f"📊 Résultats sauvegardés dans : runs/detect/senchess_colab_training")
print(f"🏆 Meilleur modèle : runs/detect/senchess_colab_training/weights/best.pt")

In [ ]:
# 💾 Télécharger le modèle entraîné
from google.colab import files
import os

# Vérifier que l'entraînement a bien eu lieu
model_dir = 'runs/detect/senchess_colab_training'
if not os.path.exists(model_dir):
    print("❌ Aucun modèle entraîné trouvé.")
    print("   Veuillez d'abord exécuter la cellule d'entraînement (cellule 7).")
else:
    print("📦 Compression du modèle entraîné...")
    !zip -r trained_model.zip runs/detect/senchess_colab_training
    
    print("⬇️ Téléchargement du fichier...")
    files.download('trained_model.zip')
    
    print("✅ Téléchargement terminé !")
    print("📂 Le fichier contient :")
    print("   - weights/best.pt : meilleur modèle")
    print("   - weights/last.pt : dernier checkpoint")
    print("   - résultats et graphiques d'entraînement")

In [ ]:
# 🧪 Tester le modèle entraîné
from ultralytics import YOLO
from google.colab import files
import os
from PIL import Image
import matplotlib.pyplot as plt

# Charger le meilleur modèle
model_path = 'runs/detect/senchess_colab_training/weights/best.pt'

if not os.path.exists(model_path):
    print("❌ Modèle non trouvé. Veuillez d'abord entraîner le modèle (cellule 7).")
else:
    model = YOLO(model_path)
    
    # Option 1 : Tester sur une image du projet (si disponible)
    test_image = 'data/chess_ultimate_1693/test/images'
    if os.path.exists(test_image):
        test_images = os.listdir(test_image)
        if test_images:
            test_img_path = os.path.join(test_image, test_images[0])
            print(f"🖼️ Test sur : {test_img_path}")
            results = model.predict(test_img_path, save=True, conf=0.25)
            print("✅ Prédiction terminée !")
            print(f"📂 Résultats sauvegardés dans : runs/detect/predict")
    
    # Option 2 : Upload une image pour tester
    print("\n📤 Ou uploadez une image de plateau d'échecs pour tester :")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"\n🔍 Analyse de {filename}...")
        results = model.predict(filename, save=True, conf=0.25)
        
        # Afficher les résultats
        img = Image.open(f'runs/detect/predict/{filename}')
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'Détections sur {filename}')
        plt.show()
        
        print(f"✅ {len(results[0].boxes)} pièces détectées !")

## 📊 Visualiser les métriques d'entraînement

Après l'entraînement, vous pouvez visualiser les performances du modèle.

In [ ]:
# 📈 Afficher les graphiques d'entraînement
from PIL import Image
import matplotlib.pyplot as plt
import os

results_dir = 'runs/detect/senchess_colab_training'

if os.path.exists(results_dir):
    # Afficher les courbes d'entraînement
    plots = ['results.png', 'confusion_matrix.png', 'F1_curve.png', 'PR_curve.png']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.ravel()
    
    for idx, plot_name in enumerate(plots):
        plot_path = os.path.join(results_dir, plot_name)
        if os.path.exists(plot_path):
            img = Image.open(plot_path)
            axes[idx].imshow(img)
            axes[idx].set_title(plot_name.replace('_', ' ').replace('.png', ''))
            axes[idx].axis('off')
        else:
            axes[idx].text(0.5, 0.5, f'{plot_name} non disponible', 
                          ha='center', va='center')
            axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Afficher les métriques finales
    print("\n📊 Résumé des performances :")
    print(f"📂 Dossier des résultats : {results_dir}")
    print("✅ Consultez results.csv pour les métriques détaillées")
else:
    print("❌ Aucun résultat d'entraînement trouvé.")
    print("   Veuillez d'abord entraîner le modèle.")